In [0]:
import tensorflow as tf
from tensorflow.keras import models, layers


In [1]:
def MobileNet():
    model = models.Sequential()
    model.add(layers.Conv2D(32, 3, strides=2, padding='same', input_shape=(224, 224, 3)))
    model.add(layers.DepthwiseConv2D(32, 1,padding='same'))
    model.add(layers.Conv2D(64, (1, 1), strides=1, padding='same'))
    model.add(layers.DepthwiseConv2D(64, 2, padding='same'))
    model.add(layers.Conv2D(128, 1, strides=1, padding='same'))
    model.add(layers.DepthwiseConv2D(128, 1,padding='same'))

    model.add(layers.Conv2D(128, (1, 1), strides=1, padding='same'))
    model.add(layers.DepthwiseConv2D(128, 2,padding='same'))
    model.add(layers.Conv2D(256, (1, 1), strides=1, padding='same'))
    model.add(layers.DepthwiseConv2D(256, 1,padding='same'))
    model.add(layers.Conv2D(256, (1, 1), strides=1, padding='same'))
    model.add(layers.DepthwiseConv2D(256, 2,padding='same'))

    model.add(layers.Conv2D(512, (1, 1), strides=1, padding='same'))
    model.add(layers.DepthwiseConv2D(512, 1,padding='same'))
    model.add(layers.Conv2D(512, (1, 1), strides=1, padding='same'))
    model.add(layers.DepthwiseConv2D(512, 1,padding='same'))
    model.add(layers.Conv2D(512, (1, 1), strides=1, padding='same'))
    model.add(layers.DepthwiseConv2D(512, 1,padding='same'))
    model.add(layers.Conv2D(512, (1, 1), strides=1, padding='same'))
    model.add(layers.DepthwiseConv2D(512, 1,padding='same'))
    model.add(layers.Conv2D(512, (1, 1), strides=1, padding='same'))
    model.add(layers.DepthwiseConv2D(512, 1,padding='same'))
    model.add(layers.Conv2D(512, (1, 1), strides=1, padding='same'))
    model.add(layers.DepthwiseConv2D(512, 2,padding='same'))

    model.add(layers.Conv2D(1024, (1, 1), strides=1, padding='same'))
    model.add(layers.DepthwiseConv2D(1024, 1,padding='same'))
    model.add(layers.Conv2D(1024, (1, 1), strides=1, padding='same'))
    model.add(layers.AveragePooling2D(pool_size=(7,7), strides=1, padding="valid"))
    model.add(layers.Dense(1024, activation=tf.nn.relu))
    model.add(layers.Dense(1000, activation=tf.nn.softmax))
    return model

In [0]:
MobileNet().summary()

Model: "sequential_22"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_146 (Conv2D)          (None, 112, 112, 32)      896       
_________________________________________________________________
depthwise_conv2d_127 (Depthw (None, 112, 112, 32)      32800     
_________________________________________________________________
conv2d_147 (Conv2D)          (None, 112, 112, 64)      2112      
_________________________________________________________________
depthwise_conv2d_128 (Depthw (None, 56, 56, 64)        262208    
_________________________________________________________________
conv2d_148 (Conv2D)          (None, 56, 56, 128)       8320      
_________________________________________________________________
depthwise_conv2d_129 (Depthw (None, 56, 56, 128)       2097280   
_________________________________________________________________
conv2d_149 (Conv2D)          (None, 56, 56, 128)     

https://blog.csdn.net/u011974639/article/details/79199306

https://arxiv.org/pdf/1704.04861.pdf

https://keras-zh.readthedocs.io/layers/convolutional/

In [0]:
def get_conv_block(tensor, channels, strides, alpha=1.0, name=''):
    channels = int(channels * alpha)

    x = layers.Conv2D(channels,
               kernel_size=(3, 3),
               strides=strides,
               padding='same',
               name='{}_conv'.format(name))(tensor)
    x = layers.BatchNormalization(name='{}_bn'.format(name))(x)
    x = layers.Activation('relu', name='{}_act'.format(name))(x)
    return x


def get_dw_sep_block(tensor, channels, strides, alpha=1.0, name=''):
    """Depthwise separable conv: A Depthwise conv followed by a Pointwise conv."""
    channels = int(channels * alpha)

    # Depthwise
    x = layers.DepthwiseConv2D(kernel_size=(3, 3),
                        strides=strides,
                        use_bias=False,
                        padding='same',
                        name='{}_dw'.format(name))(tensor)
    x = layers.BatchNormalization(name='{}_bn1'.format(name))(x)
    x = layers.Activation('relu', name='{}_act1'.format(name))(x)

    # Pointwise
    x = layers.Conv2D(channels,
               kernel_size=(1, 1),
               strides=(1, 1),
               use_bias=False,
               padding='same',
               name='{}_pw'.format(name))(x)
    x = layers.BatchNormalization(name='{}_bn2'.format(name))(x)
    x = layers.Activation('relu', name='{}_act2'.format(name))(x)
    return x

def MobileNet(shape, num_classes, alpha=1.0, include_top=True, weights=None):
    x_in = layers.Input(shape=shape)

    x = get_conv_block(x_in, 32, (2, 2), alpha=alpha, name='initial')

    layer_shape = [
        (64, (1, 1)),
        (128, (2, 2)),
        (128, (1, 1)),
        (256, (2, 2)),
        (256, (1, 1)),
        (512, (2, 2)),
        *[(512, (1, 1)) for _ in range(5)],
        (1024, (2, 2)),
        (1024, (2, 2))
    ]

    for i, (channels, strides) in enumerate(layer_shape):
        x = get_dw_sep_block(x, channels, strides, alpha=alpha, name='block{}'.format(i))

    if include_top:
        x = layers.GlobalAvgPool2D(name='global_avg')(x)
        x = layers.Dense(num_classes, activation='softmax', name='softmax')(x)

    model = models.Model(inputs=x_in, outputs=x)

    if weights is not None:
        model.load_weights(weights, by_name=True)

    return model

In [20]:
MobileNet((32,32,3), 10).summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_7 (InputLayer)         [(None, 32, 32, 3)]       0         
_________________________________________________________________
initial_conv (Conv2D)        (None, 16, 16, 32)        896       
_________________________________________________________________
initial_bn (BatchNormalizati (None, 16, 16, 32)        128       
_________________________________________________________________
initial_act (Activation)     (None, 16, 16, 32)        0         
_________________________________________________________________
block0_dw (DepthwiseConv2D)  (None, 16, 16, 32)        288       
_________________________________________________________________
block0_bn1 (BatchNormalizati (None, 16, 16, 32)        128       
_________________________________________________________________
block0_act1 (Activation)     (None, 16, 16, 32)        0     

In [0]:
def mobilenetv1(shape, num_classes, alpha=1.0):    
    model = models.Sequential()
    model.add(layers.InputLayer(input_shape=shape))

    def add_conv_block(channels, strides, kernel_size=3):
        channels = int(channels * alpha)
        model.add(layers.Conv2D(channels, kernel_size=kernel_size, use_bias=False, padding='same'))
        model.add(layers.BatchNormalization())
        model.add(layers.ReLU())
    
    def add_dw_sep_block(channels, strides):
        channels = int(channels * alpha)
        model.add(layers.DepthwiseConv2D(kernel_size=3, strides=strides, use_bias=False, padding='same'))
        model.add(layers.BatchNormalization())
        model.add(layers.ReLU())
        add_conv_block(channels, strides=1, kernel_size=1)
    
    add_conv_block(32, 2)

    model_shapes_channel_strides = [
              (64, 1),
              (128, 2),
              (128, 1),
              (256, 2),
              (256, 1),
              (512, 2),
              *[(512, 1) for _ in range(5)],
              (1024, 2),
              (1024, 2)
    ]

    for c, s in model_shapes_channel_strides:
        add_dw_sep_block(c, s)

    model.add(layers.GlobalAvgPool2D())
    model.add(layers.Dense(num_classes, activation='softmax', name='softmax'))

    return model

In [8]:
mobilenetv1((32, 32, 3), 10).summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 32, 32, 32)        864       
_________________________________________________________________
batch_normalization_1 (Batch (None, 32, 32, 32)        128       
_________________________________________________________________
re_lu_1 (ReLU)               (None, 32, 32, 32)        0         
_________________________________________________________________
depthwise_conv2d (DepthwiseC (None, 32, 32, 32)        288       
_________________________________________________________________
batch_normalization_2 (Batch (None, 32, 32, 32)        128       
_________________________________________________________________
re_lu_2 (ReLU)               (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 32, 32, 64)       